# **2) Ingest & Tiling**

In [1]:
from pathlib import Path
import os, sys, platform, json, subprocess

In [2]:
ROOT = Path.cwd().resolve().parents[0]

if (ROOT / "src").exists():
    sys.path.insert(0, str(ROOT))

print("Project root:", ROOT)
print("Python:", sys.version)
print("OS:", platform.platform())

Project root: D:\IIT BBS\Job Resources\fiolabs\diagram-intel
Python: 3.11.13 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:03:15) [MSC v.1929 64 bit (AMD64)]
OS: Windows-10-10.0.26100-SP0


In [3]:
from src.config.loader import load_cfg
cfg = load_cfg()
cfg.paths.raw, cfg.paths.interim, cfg.runtime.dpi

('D:\\IIT BBS\\Job Resources\\fiolabs\\diagram-intel\\data\\raw',
 'D:\\IIT BBS\\Job Resources\\fiolabs\\diagram-intel\\data\\interim',
 900)

In [4]:
inp_dir = Path(cfg.paths.input_pdfs)
print("Input dir:", inp_dir)
print("Exists:", inp_dir.exists())

# case-insensitive *.pdf on Windows
pdfs = [p for p in inp_dir.glob("*") if p.suffix.lower() == ".pdf"]
pdfs

Input dir: D:\IIT BBS\Job Resources\fiolabs\diagram-intel\data\input_pdfs
Exists: True


[WindowsPath('D:/IIT BBS/Job Resources/fiolabs/diagram-intel/data/input_pdfs/wiring diagram_Main Distribution Panel_Fio Labs.pdf')]

In [5]:
# Export SVG+PNG for the first PDF
from src.ingest.pdf_to_svg import extract_svg
assert len(pdfs)>0, "Put at least one drawing in data/input_pdfs/"
extract_svg(str(pdfs[0]), cfg)

2025-08-27 03:42:34.706 | INFO     | src.ingest.pdf_to_svg:extract_svg:16 - [ingest] Exporting SVG/PNG for wiring diagram_Main Distribution Panel_Fio Labs.pdf (dpi=900)
2025-08-27 03:42:37.240 | INFO     | src.ingest.pdf_to_svg:extract_svg:30 - [ingest] Engine=pymupdf | pages=1 | vector-like pages=1
2025-08-27 03:42:37.240 | INFO     | src.ingest.pdf_to_svg:extract_svg:31 - [ingest] Wrote manifest → D:\IIT BBS\Job Resources\fiolabs\diagram-intel\data\raw\manifests\wiring diagram_Main Distribution Panel_Fio Labs.json


In [6]:
# Tiler
from src.ingest.tiler import tile_pages
tile_pages(cfg)

from src.utils.io import read_json
from pathlib import Path
tile_idx = read_json(Path(cfg.paths.interim)/"tiles"/"tile_index.json")
len(tile_idx), tile_idx[:3]


2025-08-27 03:43:09.453 | INFO     | src.ingest.tiler:tile_pages:66 - [tiler] wiring diagram_Main Distribution Panel_Fio Labs page-1: wrote 602 tiles
2025-08-27 03:43:09.464 | INFO     | src.ingest.tiler:tile_pages:69 - [tiler] Wrote tile index (602 rows) → D:\IIT BBS\Job Resources\fiolabs\diagram-intel\data\interim\tiles\tile_index.json


(602,
 [{'pdf': 'wiring diagram_Main Distribution Panel_Fio Labs',
   'page': 1,
   'scale': 'micro',
   'row': 0,
   'col': 0,
   'bbox': [0, 0, 512, 512],
   'path': 'D:\\IIT BBS\\Job Resources\\fiolabs\\diagram-intel\\data\\interim\\tiles\\micro\\wiring diagram_Main Distribution Panel_Fio Labs\\page-1\\tile_r000_c000.png'},
  {'pdf': 'wiring diagram_Main Distribution Panel_Fio Labs',
   'page': 1,
   'scale': 'micro',
   'row': 0,
   'col': 1,
   'bbox': [435, 0, 947, 512],
   'path': 'D:\\IIT BBS\\Job Resources\\fiolabs\\diagram-intel\\data\\interim\\tiles\\micro\\wiring diagram_Main Distribution Panel_Fio Labs\\page-1\\tile_r000_c001.png'},
  {'pdf': 'wiring diagram_Main Distribution Panel_Fio Labs',
   'page': 1,
   'scale': 'micro',
   'row': 0,
   'col': 2,
   'bbox': [870, 0, 1382, 512],
   'path': 'D:\\IIT BBS\\Job Resources\\fiolabs\\diagram-intel\\data\\interim\\tiles\\micro\\wiring diagram_Main Distribution Panel_Fio Labs\\page-1\\tile_r000_c002.png'}])